# Scrape

In [150]:
import pandas as pd 

tourist_destinations_baguio_df = pd.read_csv(r'../csv/tourist_destinations_baguio.csv')
tourist_destinations_baguio_df = tourist_destinations_baguio_df.sort_values(['total_reviews'], ascending=[0]).reset_index(drop=True)
tourist_destinations_baguio_df

,place,description,total_reviews
0,Burnham Park,Urban parkland with a skating rink,23178
1,Mines View Observation Deck,Popular destination for scenic views,10493
2,Baguio City Market,Market for food & housewares,8382
3,Baguio Night Market,"Extensive, late-night street market",6307
4,Wright Park,Leafy spot for walks & horseback riding,5743
...,...,...,...
77,19th Tee,NaN,2
78,Asin Road Woodcarver's Village,NaN,2
79,Maryhurst Chapel - Maryhurst Road,NaN,1
80,Pool of Pines,NaN,1


In [151]:
place_list = tourist_destinations_baguio_df["place"][:20]
place_list

0                                          Burnham Park
1                           Mines View Observation Deck
2                                    Baguio City Market
3                                   Baguio Night Market
4                                           Wright Park
5                               Baguio Botanical Garden
6     Our Lady of the Atonement Cathedral (Baguio Ca...
7       Strawberry Farm - Home of Giant Strawberry Cake
8                            The Manor at Camp John Hay
9                                           Lion’s Head
10                                        BenCab Museum
11                 Heritage Hill and Nature Park Garden
12                                     Tam-awan Village
13                                 Igorot Stone Kingdom
14                Baguio Country Club Christmas Village
15                                          Bell Church
16                                Good Shepherd Convent
17                            Tree Top Adventure

In [207]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

s = Service("../tools/chromedriver.exe")
browser = webdriver.Chrome(service = s)

html_source_dict = {}

for place in place_list:
#for place in [place_list[0], place_list[1]]:
    URL_LINK = f"https://www.google.com/search?q={place}%2C+Baguio+City&hl=en-PH&gl=ph&sxsrf=ALiCzsa6LBioHP276Sw-0F31TiI4SCrd7w%3A1670351806771&ei=vouPY5vXLob3wAOl1ZfgCA&ved=0ahUKEwibrt6w0eX7AhWGO3AKHaXqBYwQ4dUDCA8&uact=5&oq=Heritage+Hill+and+Nature+Park+Garden%2C+Baguio+City&gs_lcp=Cgxnd3Mtd2l6LXNlcnAQAzICCCY6CggAEEcQ1gQQsAM6BAgjECc6CgguEK8BEMcBEEM6BggAEBYQHjoFCAAQhgM6BQghEKABSgQIQRgASgQIRhgAUJ4FWK8iYOckaARwAXgAgAHUBIgB7hSSAQswLjguMi4wLjEuMZgBAKABAcgBCMABAQ&sclient=gws-wiz-serp"
    browser.get(URL_LINK)
    time.sleep(2)

    wait = WebDriverWait(browser, 10)

    # Wait until the element has loaded successfully. May not load and output an error when connection is interrupted, simply rerun the cell.
    wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'hqzQac')))  
    browser.find_element(By.CLASS_NAME, 'hqzQac').click()
    time.sleep(2)

    times_to_scroll = (tourist_destinations_baguio_df.query(f'place == "{place}"')['total_reviews'].iloc[0] / 10) + 1
    print(f"Estimated times to scroll ({place}): {int(times_to_scroll)}")

    # Change to times_to_scroll if going for full review scraping
    for x in range(50):
        try:
            scrollable_div = browser.find_element(By.CLASS_NAME, 'review-dialog-list')
            browser.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
            time.sleep(3)
        except:
            print("Could not find review-dialog-list. Skipping...")
            break;
        

    SOURCE_HTML = browser.page_source
    soup = BeautifulSoup(SOURCE_HTML, "html.parser")
    review_posts_raw = soup.find_all('div', {"class" : "Jtu6Td"})
        
    html_source_dict[str(place)] = []
    html_source_dict[str(place)].append(review_posts_raw)
    

browser.quit()

Estimated times to scroll (Burnham Park): 2318
Estimated times to scroll (Mines View Observation Deck): 1050
Estimated times to scroll (Baguio City Market): 839
Estimated times to scroll (Baguio Night Market): 631
Estimated times to scroll (Wright Park): 575
Estimated times to scroll (Baguio Botanical Garden): 519
Estimated times to scroll (Our Lady of the Atonement Cathedral (Baguio Cathedral) - Archdiocese of Nueva Segovia): 364
Estimated times to scroll (Strawberry Farm - Home of Giant Strawberry Cake): 358
Estimated times to scroll (The Manor at Camp John Hay): 312
Could not find review-dialog-list. Skipping...
Estimated times to scroll (Lion’s Head): 280
Estimated times to scroll (BenCab Museum): 276
Estimated times to scroll (Heritage Hill and Nature Park Garden): 215
Estimated times to scroll (Tam-awan Village): 189
Estimated times to scroll (Igorot Stone Kingdom): 185
Estimated times to scroll (Baguio Country Club Christmas Village): 148
Estimated times to scroll (Bell Church):

In [228]:
# Instantiate inner list for destination place's review posts
review_posts_dict = {'tourist_destination': [], 'review_post' : []}

for key in html_source_dict:
    bs_html_raw = BeautifulSoup(str(html_source_dict[key]))
    review_post_list_raw = bs_html_raw.find_all('span', {"jscontroller" : "MZnM8e"})

    for review_post_element in review_post_list_raw:
        review_posts_dict['tourist_destination'].append(str(key))

        try:
            review_post_element.find("div", {"class" : "k8MTF"}).decompose()
        except Exception:
            pass

        try:
            review_full_post = review_post_element.find('span', {"class" : "review-full-text"})
            review_posts_dict['review_post'].append(review_full_post.get_text())
            #print(review_full_post.get_text())
        except Exception as e:
            review_posts_dict['review_post'].append(review_post_element.get_text())
            #print(review_post_element.get_text())


destinations_set = set(review_posts_dict["tourist_destination"])
unique_destinations_list = (list(destinations_set))

for destination in unique_destinations_list:
    print(f"{destination} :", review_posts_dict['tourist_destination'].count(destination))

Tree Top Adventure Baguio : 510
Wright Park : 510
Baguio Night Market : 510
Lion’s Head : 110
Strawberry Farm - Home of Giant Strawberry Cake : 510
Baguio Botanical Garden : 510
Burnham Park : 510
Our Lady of the Atonement Cathedral (Baguio Cathedral) - Archdiocese of Nueva Segovia : 510
Baguio City Market : 510
Mines View Observation Deck : 510
BenCab Museum : 510
Valley of Colors : 220
Igorot Stone Kingdom : 510
Teacher's Camp : 40
Baguio Country Club Christmas Village : 510
Heritage Hill and Nature Park Garden : 510
Good Shepherd Convent : 510
Tam-awan Village : 510


# Preprocess Preliminaries

In [270]:
import numpy as np

review_posts_df = pd.DataFrame.from_dict(review_posts_dict)

# Remove "(Translated by Google)", "(Original)", and succeeding original review post after (Original)
review_posts_df['review_post'] = review_posts_df['review_post'].map(lambda x: x.replace('(Translated by Google) ', ''))
review_posts_df['review_post'] = review_posts_df['review_post'].map(lambda x: x.split('(Original)', 1)[0])

# Replace empty cells with NaN values
review_posts_df['review_post'].replace('', np.nan, inplace = True)
review_posts_df.isnull().sum()


tourist_destination      0
review_post            197
dtype: int64

In [271]:
review_posts_df.dropna(inplace = True)
review_posts_df.isnull().sum()

tourist_destination    0
review_post            0
dtype: int64

In [272]:
review_posts_df.to_csv("../csv/uncleaned/tourist_destinations_review_posts_baguio.csv", index = False)
review_posts_df

,tourist_destination,review_post
0,Burnham Park,An excellent place for family and friends to v...
1,Burnham Park,A lovely place to be on a nice day.We were ple...
2,Burnham Park,Definitely one of the best places to visit in ...
3,Burnham Park,I have always been looking at this park ever s...
4,Burnham Park,Very beautiful park with boating and cycling a...
...,...,...
8015,Teacher's Camp,Place is quite old but still works very well. ...
8016,Teacher's Camp,Not crowded. Lots of open space.
8017,Teacher's Camp,Some staff are not approachable when i went to...
8018,Teacher's Camp,Very very affordable place. Nice and cozy. Kak...
